In [ ]:
import tensorflow as tf
print(tf.__version__)

import keras
from keras.layers import Input, Dense
from keras.layers import Flatten, Reshape, LSTM
from keras.models import Model, Sequential
from keras.datasets import mnist
from keras.losses import binary_crossentropy
from keras import backend as K
from keras.optimizers import RMSprop, Adam

import numpy as np
import matplotlib.pyplot as plt 
import cv2
import skimage
import imutils
%matplotlib inline

np.random.seed(42)

In [ ]:
# load data form npy file
data_time = np.load("/content/drive/MyDrive/stats402_data/latent_space_representation_gray_evolutionL8_cropped.npy")

In [ ]:
x = data_time[:, 0]
y = data_time[:, 1::]
x = np.reshape(x, (x.shape[0], 1))

train_sample = 3500
x_train = x[0:train_sample, :]
y_train = y[0:train_sample, :]

# reshape x for training 
x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))

print(x_train.shape)
print(y_train.shape)

In [ ]:
step = 1

model_LSTM = Sequential()
model_LSTM.add(LSTM(128, input_shape=(1, step), return_sequences=True, activation='relu'))
model_LSTM.add(LSTM(128, return_sequences=True, activation='relu'))
model_LSTM.add(LSTM(256, return_sequences=True, activation='relu'))
model_LSTM.add(LSTM(256, activation='relu'))
model_LSTM.add(Dense(y_train.shape[1]))

model_LSTM.compile(loss='mean_squared_error', optimizer=Adam(lr=0.00001))
model_LSTM.summary()

In [ ]:
epochs = 400
batch_size = 128

history = model_LSTM.fit(x_train, y_train,
                    epochs = epochs,
                    batch_size = batch_size)

In [ ]:
# plot the traning data and the prediction 

num = 6000

x_range_train = np.arange(3500)
x_range_extend = np.arange(num)

x_range_extend_reshape = np.reshape(x_range_extend, (num, 1, 1))

y_pred_extend = model_LSTM.predict(x_range_extend_reshape)


plt.figure(figsize=(10, 20))

for i in range(8):
    plt.subplot(8, 1, i+1)
    plt.plot(x_range_train, y_train[:,i], 'b', label='training')
    plt.plot(x_range_extend, y_pred_extend[:,i], 'g', label='predicted')
    plt.legend()
    plt.xlabel('x')
    plt.ylabel('y')

plt.show()


In [ ]:
# save the predicted representations to npy file

np.save("/content/drive/MyDrive/stats402_data/predicted_envolving_large_representation_B_channel.npy", y_pred_extend)